-  spark UI 

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.appName('uiapp').master('local').getOrCreate()

In [3]:
data=[('a',2),('b',5)]
df=spark.createDataFrame(data=data,schema=['alfabet','value'])
df.show()

+-------+-----+
|alfabet|value|
+-------+-----+
|      a|    2|
|      b|    5|
+-------+-----+



## converting to Pandas dataframe

In [4]:
pandaDF=df.toPandas()

In [5]:
pandaDF.head()

,alfabet,value
0,a,2
1,b,5


## transforming data to json

In [7]:
df.toJSON().collect()

['{"alfabet":"a","value":2}', '{"alfabet":"b","value":5}']

## spark default shuffle partions

In [8]:
spark.conf.get('spark.sql.shuffle.partitions')

'200'

In [9]:
df.explain()

== Physical Plan ==
*(1) Scan ExistingRDD[alfabet#0,value#1L]




In [10]:
spark.conf.set("spark.sql.shuffle.partitions", "5")

In [11]:
spark.conf.get('spark.sql.shuffle.partitions')

'5'

## using dataframe like table 

In [12]:
df.createOrReplaceTempView('Table1')

In [13]:
spark.sql("SELECT * FROM Table1").show()

+-------+-----+
|alfabet|value|
+-------+-----+
|      a|    2|
|      b|    5|
+-------+-----+



In [14]:
spark.sql("SELECT alfabet,value FROM Table1 where value< 5 ").show()

+-------+-----+
|alfabet|value|
+-------+-----+
|      a|    2|
+-------+-----+



In [27]:
path='D:\RetailData\Features data set.csv'
retailDF=spark.read.option('inferSchema','true').option('header','True').csv(path)

In [28]:
retailDF.show(4)

+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|Store|      Date|Temperature|Fuel_Price|MarkDown1|MarkDown2|MarkDown3|MarkDown4|MarkDown5|        CPI|Unemployment|IsHoliday|
+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+------------+---------+
|    1|05/02/2010|      42.31|     2.572|       NA|       NA|       NA|       NA|       NA|211.0963582|       8.106|    false|
|    1|12/02/2010|      38.51|     2.548|       NA|       NA|       NA|       NA|       NA|211.2421698|       8.106|     true|
|    1|19/02/2010|      39.93|     2.514|       NA|       NA|       NA|       NA|       NA|211.2891429|       8.106|    false|
|    1|26/02/2010|      46.63|     2.561|       NA|       NA|       NA|       NA|       NA|211.3196429|       8.106|    false|
+-----+----------+-----------+----------+---------+---------+---------+---------+---------+-----------+--------

In [29]:
from pyspark.sql.functions import col,isnan, when, count
retailDF.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show()

AnalysisException: cannot resolve 'alfabet' given input columns: [CPI, Date, Fuel_Price, IsHoliday, MarkDown1, MarkDown2, MarkDown3, MarkDown4, MarkDown5, Store, Temperature, Unemployment];
'Aggregate [count(CASE WHEN (isnan('alfabet) OR isnull('alfabet)) THEN alfabet END) AS alfabet#270, count(CASE WHEN (isnan('value) OR isnull('value)) THEN value END) AS value#272]
+- Relation [Store#184,Date#185,Temperature#186,Fuel_Price#187,MarkDown1#188,MarkDown2#189,MarkDown3#190,MarkDown4#191,MarkDown5#192,CPI#193,Unemployment#194,IsHoliday#195] csv
